FileNotFoundError: [Errno 2] No such file or directory: 'brown/'

In [131]:
import pycrfsuite
#from bs4 import BeautifulSoup as bs
#from bs4.element import Tag
#import codecs
#import nltk
from nltk import word_tokenize, pos_tag
#from nltk.corpus import wordnet as wn
from sklearn.model_selection import train_test_split
import pycrfsuite
import os, os.path, sys
import glob
from xml.etree import ElementTree
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import  confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import XMLExtractor as ex
from nltk.parse.stanford import StanfordDependencyParser
#import spacy - this uses deeplearning, don't use it ;)

from sklearn.feature_extraction.text import  CountVectorizer

from scipy.spatial.distance import cosine
from itertools import chain

In [5]:
## helper functions 
def remov_punct(withpunct):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    without_punct = ""
    char = 'nan'
    for char in withpunct:
        if char not in punctuations:
            without_punct = without_punct + char
    return(without_punct)
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [text[-1] for text in doc]

In [6]:
"""
def append_annotations(files):
    xml_files = glob.glob(files +"/*.xml")
    xml_element_tree = None
    new_data = ""
    for xml_file in xml_files:
        data = ElementTree.parse(xml_file).getroot()
        #print ElementTree.tostring(data)        
        temp = ElementTree.tostring(data)
        new_data += (temp)
    return(new_data)

#this function removes special characters and punctuations
def remov_punct(withpunct):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    without_punct = ""
    char = 'nan'
    for char in withpunct:
        if char not in punctuations:
            without_punct = without_punct + char
    return(without_punct)
"""


TypeError: word_tokenize() missing 1 required positional argument: 'text'

In [7]:
## Let's Chechk our baseline approach
def get_aspect_list(path):

    aspect_count_b = set()
    aspect_count_i = set()
    for elem in ElementTree.parse(path).getroot().findall('sentence'):

        aspect_terms = [word_tokenize(e.attrib['term']) for es in elem.findall('aspectTerms') for e in es if es is not None]

        for aspect in aspect_terms:
            for i in range(len(aspect)):
                token = aspect[i]
                if i == 0:
                    aspect_count_b.add(token)
                else:
                    aspect_count_i.add(token)

    
    

    return aspect_count_b, aspect_count_i

In [8]:

# get corpus for our baseline
def get_corpus_for_baseline(path):
    docs = []
    
    for elem in ElementTree.parse(path).getroot().findall('sentence'):
        texts = []
        text = elem.find('text').text
        
        tokens = word_tokenize(text)
        

        
        id = elem.get('id')
        aspect_terms = [e.attrib['term'] for es in elem.findall('aspectTerms') for e in es if es is not None]

        for w in tokens:
            
            label = "O"
            for aspect in aspect_terms:
            
                aspect_parts = aspect.split(" ")
                for i in range(len(aspect_parts)):
                    part = aspect_parts[i]
                    if w == part.replace("\"","") and i == 0:
                        label = "B"
                    elif w == part:
                        label = "I"
            

            
            texts.append((w, label))
        docs.append(texts)
    

    return docs

In [9]:
def run_benchmark(aspect_b, aspect_i,test):
    y_pred = []
    y_true = []
    for query in test:
        current_pred = []
        current_true = []
        on_b = False
        for token, label in query:
            if on_b is False:
                if token in aspect_b:
                    on_b = True
                    current_pred.append("B")
                    current_true.append(label)
                else:
                    current_pred.append("O")
                    current_true.append(label)
            elif on_b is True:
                if token in aspect_i:
                    current_pred.append("I")
                    current_true.append(label)
                elif token in aspect_b:
                    current_pred.append("B")
                    current_true.append(label)
                else:
                    current_pred.append("O")
                    current_true.append(label)
                    on_b = False
        y_pred.append(current_pred)
        y_true.append(current_true)        
    return y_true, y_pred
                    
            

In [13]:
aspect_b, aspects_i = get_aspect_list('data/Laptops_Train_v2.xml')
test= get_corpus_for_baseline('data/Laptops_Test_Gold.xml')
y_true, y_pred = run_benchmark(aspect_b, aspects_i,test)


In [14]:
labels = {"B": 0, "I": 1, "O":2}

# Convert the sequences of tags into a 1-dimensional array
y_pred = np.array([labels[tag] for row in y_pred for tag in row])
y_true = np.array([labels[tag] for row in y_true for tag in row])

# Print out the classification report for basline. It will not be very good
print(classification_report(
    y_true, y_pred,
    target_names=["B", "I", "O"]))

             precision    recall  f1-score   support

          B       0.41      0.69      0.51       651
          I       0.28      0.44      0.34       408
          O       0.97      0.91      0.94     10741

avg / total       0.91      0.88      0.89     11800



In [15]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Compute confusion matrix
cnf_matrix = confusion_matrix(y_true, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=[0,1,2],
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=[0,1,2], normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[ 450   37  164]
 [  80  181  147]
 [ 576  426 9739]]
Normalized confusion matrix
[[0.69 0.06 0.25]
 [0.2  0.44 0.36]
 [0.05 0.04 0.91]]


In [ ]:


def get_corpus(path):
    docs = []
    word_to_cluster = {word:cluster[-k:] for cluster, size, words in clusters for word in words}
    
    for elem in ElementTree.parse(path).getroot().findall('sentence'):
        texts = []
        text = elem.find('text').text
        
        tokens = word_tokenize(text)
        tokens_pos = nltk.pos_tag(tokens)

        
        id = elem.get('id')
        aspect_terms = [e.attrib['term'] for es in elem.findall('aspectTerms') for e in es if es is not None]

    # Loop through each child of the element under "textwithnamedentities"
    
        for w,pos in tokens_pos:
            
            label = "O"
            for aspect in aspect_terms:
            
                aspect_parts = aspect.split(" ")
                for i in range(len(aspect_parts)):
                    part = aspect_parts[i]
                    if w == part.replace("\"","") and i == 0:
                        label = "B"
                    elif w == part:
                        label = "I"
            


            texts.append((w, pos ,  label))
        docs.append(texts)
    

    return docs

In [187]:
def word2features(doc, i,bias=True, lower=True,cluster=True, minus_2=True, minus_3=True,upper=True,hasnumber=True,postag=True,postag2=True,title=True,plus_2=True,plus_3=True, before_n=1, after_n=1 ):
    
    word = doc[i][0]
    
    pos = doc[i][1]
    if cluster:
        cluster = doc[i][2]

         
    features = [
        
        
    ]
    if bias:
        features.append('bias')
    if lower:
        features.append('word.lower='+word.lower())
    if minus_2:
        features.append('word[-2:]='+  word[-2:].lower())
    if minus_3:
        features.append('word[-3:]='+  word[-3:].lower())
    if upper:
        features.append('word.isupper=%s' % word.isupper())
    if hasnumber:
        features.append('word.hasnumber=%s' % any(char.isdigit() for char in word))
    if postag:
        features.append('postag=' + pos)
    if postag2:
        features.append('postag2=' + pos[:2])  
    if title:
        features.append('title=%s' %word.istitle())
    if cluster:
        features.append('cluster='+cluster)
    if plus_2:
        if len(word) >=2:
            features.append('word[:2]='+word[:2].lower())
        else:
            features.append('**')
    if plus_3:
        if len(word) >=3:
            features.append('word[:3]='+word[:3].lower())
        else:
            features.append('***')
    for j in range(before_n):
        if i > j:
            word = doc[i-j][0]
            pos = doc[i-j][1]
            cluster = doc[i-j][2]

            if lower:
                features.append("-"+str(j+1)+':word.lower='+word.lower())
            if minus_2:
                features.append("-"+str(j+1)+':word[-2:]='+  word[-2:].lower())
            if minus_3:
                features.append("-"+str(j+1)+':word[-3:]='+  word[-3:].lower())
            if upper:
                features.append("-"+str(j+1)+':word.isupper=%s' % word.isupper())
            if hasnumber:
                features.append("-"+str(j+1)+':word.hasnumber=%s' % any(char.isdigit() for char in word))
            if postag:
                features.append("-"+str(j+1)+':postag=' + pos)
            if postag2:
                features.append("-"+str(j+1)+':postag2=' + pos[:2])  
            if title:
                features.append("-"+str(j+1)+':title=%s' %word.istitle())
            if cluster:
                features.append("-"+str(j+1)+'cluster='+cluster)
            if plus_2:
                if len(word) >=2:
                    features.append("-"+str(j+1)+':word[:2]='+word[:2].lower())
                else:
                    features.append("-"+str(j+1)+':**')
            if plus_3:
                if len(word) >=3:
                    features.append("-"+str(j+1)+':word[:3]='+word[:3].lower())
                else:
                    features.append("-"+str(j+1)+':***')
        else:
            features.append('BOS'+str(j+1))

    for j in range(after_n):
        if i < len(doc)-j:
            word = doc[i+j][0]
            pos = doc[i+j][1]
            cluster = doc[i+j][2]

            if lower:
                features.append("+"+str(j+1)+':word.lower='+word.lower())
            if minus_2:
                features.append("+"+str(j+1)+':word[-2:]='+  word[-2:].lower())
            if minus_3:
                features.append("+"+str(j+1)+':word[-3:]='+  word[-3:].lower())
            if upper:
                features.append("+"+str(j+1)+':word.isupper=%s' % word.isupper())
            if hasnumber:
                features.append("+"+str(j+1)+':word.hasnumber=%s' % any(char.isdigit() for char in word))
            if postag:
                features.append("+"+str(j+1)+':postag=' + pos)
            if postag2:
                features.append("+"+str(j+1)+':postag2=' + pos[:2]) 
            if title:
                features.append("+"+str(j+1)+':title=%s' %word.istitle())                
            if cluster:
                features.append("+"+str(j)+'cluster='+cluster)
            if plus_2:
                if len(word) >=2:
                    features.append("+"+str(j+1)+':word[:2]='+word[:2].lower())
                else:
                    features.append("+"+str(j+1)+':**')
            if plus_3:
                if len(word) >=3:
                    features.append("+"+str(j+1)+':word[:3]='+word[:3].lower())
                else:
                    features.append("+"+str(j+1)+':***')
        else:
            features.append('EOS'+str(j+1))


    return features

In [117]:
# Evaluate features
from sklearn.metrics import f1_score
def extract_features_2(docs , bias=True, lower=True, cluster=True, minus_2=True, minus_3=True,upper=True,hasnumber=True,postag=True,postag2=True,title=True,plus_2=True,plus_3=True, before_n=0, after_n=0 ): 

    return [ word2features(docs, i, bias=bias, lower=lower, cluster=True, minus_2=minus_2, minus_3=minus_3,upper=upper,hasnumber=hasnumber,postag=postag,postag2=postag2,title=title,plus_2=plus_2,plus_3=plus_3, before_n=before_n, after_n=after_n ) for i in range(len(docs))]


def get_labels(docs):
    return [doc[-1] for doc in docs]

def run_gridsearch():
    train = get_corpus_with_clusters('data/Laptops_Train_v2.xml',5,results)
    test = get_corpus_with_clusters('data/Laptops_Test_Gold.xml',5,results)

    runs = [[minus_2, minus_3, upper, hasnumber, postag,postag2,title,plus_2,plus_3]  for minus_2 in range(2) for minus_3 in range(2) for upper in range(2) for hasnumber in range(2) for postag in range(2) for postag2 in range(2) for title in range(2) for plus_2 in range(2) for plus_3 in range(2)]
    reports =  []
    fscores = []

    minus_2_a = [[],[]]
    minus_3_a = [[],[]]
    upper_a =  [[],[]]
    hasnumber_a =[[],[]]
    postag_a = [[],[]]
    postag2_a = [[],[]]
    title_a = [[],[]]
    plus_2_a = [[],[]]
    plus_3_a = [[],[]]
    after_n_a = [[],[],[]]
    before_n_a = [[],[],[]]
    
    for run in runs:
        bias = True
        lower = True
        minus_2 = run[0]
        minus_3 = run[1]
        upper =  run[2]
        hasnumber =run[3]
        postag = run[4]
        postag2 = run[5]
        title = run[6]
        plus_2 = run[7]
        plus_3 = run[8]
        before_n = 0
        after_n = 0
        X_train = [extract_features_2(doc, bias, lower, minus_2, minus_3,upper,hasnumber,postag,postag2,title,plus_2,plus_3, before_n, after_n ) for doc in train]
        y_train = [get_labels(doc) for doc in train]

        X_test = [extract_features_2(doc, bias, lower, minus_2, minus_3,upper,hasnumber,postag,postag2,title,plus_2,plus_3, before_n, after_n ) for doc in test]
        y_test = [get_labels(doc) for doc in test]
                                                     
        trainer = pycrfsuite.Trainer(verbose=False)

 
        for xseq, yseq in zip(X_train, y_train):
            trainer.append(xseq, yseq)


        trainer.set_params({

            'feature.possible_states': 1,
    


        })

        trainer.train('crf.model')
        tagger = pycrfsuite.Tagger()
        tagger.open('crf.model')
        y_pred = [tagger.tag(xseq) for xseq in X_test]
                                

        labels = {"B": 0, "I": 1, "O":2}

        y_pred = np.array([labels[tag] for row in y_pred for tag in row])
        y_true = np.array([labels[tag] for row in y_test for tag in row])



        report = classification_report(
            y_true, y_pred,
        target_names=["B", "I", "O"])
        reports.append(report +"\n bias={0}, lower={1}, minus_2={2}, minus_3={3},upper={4},hasnumber={5},postag={6},postag2={7},title={8},plus_2={9},plus_3={10}, before_n={11}, after_n={12}".format(bias, lower, minus_2, minus_3,upper,hasnumber,postag,postag2,title,plus_2,plus_3, before_n, after_n))
        fscore = f1_score(y_true, y_pred, average='macro')
        fscores.append(fscore)
        print(reports[-1])
        print(fscores[-1])
        if minus_2:
            minus_2_a[0].append(fscore)
        else:
            minus_2_a[1].append(fscore)
            
        if minus_3:
            minus_3_a[0].append(fscore)
        else:
            minus_3_a[1].append(fscore)
        if upper:
            upper_a[0].append(fscore)
        else:
            upper_a[1].append(fscore)                        
        if hasnumber_a:
            hasnumber_a[0].append(fscore)
        else:
            hasnumber_a[1].append(fscore)   
        if postag:
            postag_a[0].append(fscore)
        else:
            postag_a[1].append(fscore)
        if postag2:
            postag2_a[0].append(fscore)
        else:
            postag2_a[1].append(fscore)    
        if title:
            title_a[0].append(fscore)
        else:
            title_a[1].append(fscore) 
        if plus_2_a:
            plus_2_a[0].append(fscore)
        else:
            plus_2_a[1].append(fscore) 
        if plus_3:
            plus_3_a[0].append(fscore)
        else:
            plus_3_a[1].append(fscore) 
        if after_n == 2:
            after_n_a[0].append(fscore)
        elif after_n == 1:
            after_n_a[1].append(fscore) 
        else:
            before_n_a[2].append(fscore) 
        if before_n == 2:
            after_n_a[0].append(fscore)
        elif before_n == 1:
            after_n_a[1].append(fscore) 
        else:
            before_n_a[2].append(fscore)
            
    return reports, fscores, minus_2_a, minus_3_a,upper_a, hasnumber_a,postag_a,postag2_a,title_a, plus_2_a,plus_3_a, after_n_a,before_n_a


In [ ]:
reports, fscores, minus_2_a,minus_3_a,upper_a,hasnumber_a,postag_a,postag2_a,title_a,plus_2_a,plus_3_a , after_n_a,before_n_a = run_gridsearch()

In [104]:
print ( np.mean( postag_a[0]), np.mean( postag_a[1]))
print (np.mean( postag2_a[0]), np.mean( postag2_a[1]))
print (np.mean( upper_a[0]), np.mean( upper_a[1]))
print (np.mean( hasnumber_a[0]), np.mean( hasnumber_a[1]))
print (np.mean( title_a[0]), np.mean( title_a[1]))
print (np.mean( minus_2_a[0]), np.mean( minus_2_a[1]))
print (np.mean( minus_3_a[0]), np.mean( minus_3_a[1]))
print (np.mean( plus_2_a[0]), np.mean( plus_2_a[1]))
print (np.mean( plus_3_a[0]), np.mean( plus_3_a[1]))

0.7470777283613109 0.7337786712419698
0.748427261669859 0.7324291379334218
0.7403746893727258 0.740481710230555
0.7409247567108124 0.7399316428924683
0.7402299989920982 0.7406264006111825
0.7469556413155429 0.7339007582877379
0.7488526063552827 0.732003793247998
0.745076325266407 0.7357800743368735
0.7527423362569303 0.7281140633463503


In [ ]:
## further Feature engineering, Brown Clusers
# Create T table, algorithms for Brown clustering comes from this  http://www.aclweb.org/anthology/N16-1093

def make_domain_table(word_doc_table,words_in_doc_freq,sum_in_doc_freq ):
    T = np.zeros((word_doc_table.shape[0],word_doc_table.shape[0]))
    for i in range(word_doc_table.shape[0]):

        for j in range(word_doc_table.shape[0]):
            
            if i == j:
                T[i,j] = 1
                continue
            i_feat = words_in_doc_freq[:,i]
            j_feat = words_in_doc_freq[:,j]
            counts = np.dot(i_feat.T,j_feat)
            
            if counts > 0:
                val = np.log(word_doc_table.shape[1]*(counts/(sum_in_doc_freq[0,i]*sum_in_doc_freq[0,j])))/(-np.log((counts/word_doc_table.shape[1])))
            
            else:
                val = -1
            T[i,j] = val
            T[j,i] = val
    
    return T
def breaks_constraints(c_l, c_m, c_i,c_j,count_dic, C_pos_dic, T, delta):
    distances = []
    for u_id in c_l:
        u = T[u_id]
        for v_id in c_m:
            v = T[v_id]
            dist = 1- cosine(u,v)
            distances.append(dist)
    avg_dist = np.average(np.array(distances))
    r_l = T[count_dic[c_i][1],:]
    r_m = T[count_dic[c_j][1],:]
    r_dist = 1 - cosine(r_l,r_m)
    distance = max(avg_dist,r_dist)
    if distance > delta:
        return True, distance
    #if any(v in C_pos_dic[c_i] for v in ["in","ca","vbn","vbd","vbp" ,"vb" , "vbd" ,"vbz", "vbg"]) or any(v in C_pos_dic[c_j] for v in ["in","ca","vbn","vbd","vbp" , "vb" , "vbd" , "vbz", "vbg"]   ):
        
        #return True, distance
    
    if "nn" not in C_pos_dic[c_i] and "nn" not in C_pos_dic[c_j] and "nnp" not in C_pos_dic[c_i] and "nnp" not in C_pos_dic[c_j] and "nns" not in C_pos_dic[c_i] and "nns" not in C_pos_dic[c_j]:
        return True , distance

    return False, distance

def brown_clustering( s, delta, k, words_freq, T):
    words_freq_ind = {i:set([words_freq[i][2]]) for i in range(s)}
    words = {i:set([words_freq[i][0]]) for i in range(s)}
    
    theta = set([i for i in range(s)])
    C = words_freq_ind
    C_count = {i:(words_freq[i][1], words_freq[i][2] ) for i in range(s)}
    C_pos_dic = {i:set([words_freq[i][0].split("_")[-1]]) for i in range(s)}

    
    while len(theta)> 1:
        distances = []
        for c_i in theta:
            c_l= C[c_i]
            for c_j in theta:
                if c_j == c_l:
                    continue
                c_m = C[c_j]
                bol, distance = breaks_constraints(c_l,c_m, c_i,c_j,C_count,C_pos_dic,T, delta)
                if bol == False:
                    distances.append((distance,c_i,c_j))
        
        if len(distances) == 0:
            break
        distances= sorted(distances, key = lambda q: q[0])
        distance,c_i,c_j  = distances[0]
        C[c_j] = C[c_j].union(C[c_i])
        C_pos_dic[c_j]=C_pos_dic[c_j].union(C_pos_dic[c_i])
        words[c_j] = words[c_j].union(words[c_i])
        c_l_count=C_count[c_i]
        if c_l_count[0] > C_count[c_j][0]:
            C_count[c_j] = c_l_count
        theta = theta - set([c_i])
        
    words_freq_ind_not_prime ={i+s:set([words_freq[i+s][2]]) for i in range(len(words_freq[s:]))} 
    words_not_prime = {i+s:set([words_freq[i+s][0]]) for i in range(len(words_freq[s:]))}
    C_pos_dic_not_prime = {i+s:set([words_freq[i+s][0].split("_")[-1]]) for i in range(len(words_freq[s:]))}
    
    C_pos_dic = {**C_pos_dic,**C_pos_dic_not_prime}
    C_count_not_prime = {i:(words_freq[i][1],words_freq[i][2] ) for i in range(s,len(words_freq))}
    C_count = {**C_count,**C_count_not_prime}
    words = {**words, **words_not_prime}

    for ind in words_freq_ind_not_prime:
        best_distance = [1000000, None, None]
        found = False
        for c_i in theta:
            c_l = C[c_i]
            bol, distance = breaks_constraints(c_l,set([ind]), c_i,ind, C_count,C_pos_dic, T, delta)
            if bol == False and distance < best_distance[0]:
                best_distance = [distance, c_i, ind]
                found = True
        if found:
            
            C[best_distance[1]]=C[best_distance[1]].union(set([ind]))

            C_pos_dic[best_distance[1]]=C_pos_dic[best_distance[1]].union(C_pos_dic[ind])
            words[best_distance[1]]=words[best_distance[1]].union(words[ind])
         


    total_members = [(len(words[c_i]), words[c_i]) for c_i in theta]
    total_members = sorted(total_members,reverse=True)
    total_members = [('{0:08b}'.format(i),total_members[i][0], total_members[i][1] ) for i in range(len(total_members))]
    return total_members[:k]

In [17]:


def get_corpus4_vectorizing(path):
    docs = []

    for elem in ElementTree.parse(path).getroot().findall('sentence'):
        texts = []
        text = elem.find('text').text
        
        tokens = word_tokenize(text)
        tokens_pos = nltk.pos_tag(tokens)

        
        id = elem.get('id')
        aspect_terms = [e.attrib['term'] for es in elem.findall('aspectTerms') for e in es if es is not None]
        aspect_polarity = [e.attrib['polarity'] for es in elem.findall('aspectTerms') for e in es if es is not None]
        aspect_from_to = [(e.attrib['from'],e.attrib['to']) for es in elem.findall('aspectTerms') for e in es if es is not None]

    # Loop through each child of the element under "textwithnamedentities"
    
        for w,pos in tokens_pos:
            


            if w.lower() not in tx.ENGLISH_STOP_WORDS:
                texts.append((w +"_"+pos))
        final_text = ""
        for tt in texts:
            final_text += tt+" "
        
        docs.append(final_text[:-1])
    

    return docs

In [188]:
corpus = get_corpus4_vectorizing('data/Laptops_Train_v2.xml')
vectorizer = CountVectorizer()
vec = vectorizer.fit(corpus)
X = vec.transform(corpus)
sum_vec = X.sum(axis=0)
regular_vocab = vec.vocabulary_
reverse_vocab = {index:word for word, index in regular_vocab.items()}
words_freq = [[word, sum_vec[0, idx], idx] for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda q: q[1], reverse=True)
words_in_doc_freq = X > 0
words_in_doc_freq = words_in_doc_freq.astype(int).todense()
words_in_doc_sum = words_in_doc_freq.sum(axis=0)
X= X.T.toarray()


In [24]:
T = make_domain_table(X,words_in_doc_freq,words_in_doc_sum )
T = (T-np.min(T))/(np.max(T)-np.min(T))

In [105]:
results = brown_clustering( 300, .05, 128, words_freq,  T)

In [107]:

def get_corpus_with_clusters(path, k, brown_cluster=[], topics=[]):
    docs = []
    word_to_cluster = {word:brown_cluster[-k:] for brown_cluster, size, words in brown_cluster for word in words}
     
    for elem in ElementTree.parse(path).getroot().findall('sentence'):
        texts = []
        text = elem.find('text').text
        
        tokens = word_tokenize(text)
        tokens_pos = nltk.pos_tag(tokens)

        
        id = elem.get('id')
        aspect_terms = [e.attrib['term'] for es in elem.findall('aspectTerms') for e in es if es is not None]

    # Loop through each child of the element under "textwithnamedentities"
    
        for w,pos in tokens_pos:
            
            label = "O"
            for aspect in aspect_terms:
            
                aspect_parts = aspect.split(" ")
                for i in range(len(aspect_parts)):
                    part = aspect_parts[i]
                    if w == part.replace("\"","") and i == 0:
                        label = "B"
                    elif w == part:
                        label = "I"
            

            cluster = "null"
            
            if  w.lower()+"_"+pos.lower() in word_to_cluster.keys():
                
                cluster = word_to_cluster[ w.lower()+"_"+pos.lower()]
            texts.append((w, pos , cluster, label))
        docs.append(texts)
    

    return docs

In [189]:

train = get_corpus_with_clusters('data/Laptops_Train_v2.xml',4,brown_cluster= results)
test = get_corpus_with_clusters('data/Laptops_Test_Gold.xml',4,brown_cluster=results)
X_train = [extract_features_2(doc,title=False, hasnumber=False, cluster=False, after_n=2,before_n=2) for doc in train]
y_train = [get_labels(doc) for doc in train]

X_test = [extract_features_2(doc,title=False, hasnumber=False, cluster=False,after_n=2,before_n=2) for doc in test]
y_test = [get_labels(doc) for doc in test]



In [190]:

trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({

'feature.possible_states': 1,
    # maximum number of iterations


})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 1
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 113214
Seconds required: 1.543

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 2147483647
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 14536.072830
Feature norm: 1.000000
Error norm: 9937.739980
Active features: 113214
Line search trials: 1
Line search step: 0.000008
Seconds required for this iteration: 0.120

***** Iteration #2 *****
Loss: 13853.898870
Feature norm: 0.963150
Error norm: 8794.765951
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.060

***** Iteration #3 *****
Loss: 12043.854030
Feature norm: 0.920499
Error norm: 5280.471855
Active features: 113214
Line search trials: 2
Line search step: 0.389508
Seconds requ

***** Iteration #39 *****
Loss: 2499.003547
Feature norm: 18.246328
Error norm: 910.793268
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.082

***** Iteration #40 *****
Loss: 2467.080291
Feature norm: 18.683067
Error norm: 393.918808
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.058

***** Iteration #41 *****
Loss: 2447.675419
Feature norm: 18.961181
Error norm: 253.994375
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.064

***** Iteration #42 *****
Loss: 2431.115556
Feature norm: 19.181201
Error norm: 309.408344
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.078

***** Iteration #43 *****
Loss: 2399.388569
Feature norm: 19.595431
Error norm: 385.847408
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seco

***** Iteration #82 *****
Loss: 1975.159327
Feature norm: 25.020108
Error norm: 87.904106
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.055

***** Iteration #83 *****
Loss: 1973.570907
Feature norm: 25.100023
Error norm: 169.348284
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.054

***** Iteration #84 *****
Loss: 1971.433767
Feature norm: 25.102454
Error norm: 108.914492
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.063

***** Iteration #85 *****
Loss: 1967.423810
Feature norm: 25.121450
Error norm: 95.439105
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.060

***** Iteration #86 *****
Loss: 1965.596828
Feature norm: 25.142186
Error norm: 83.574829
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds

***** Iteration #123 *****
Loss: 1927.656695
Feature norm: 25.134567
Error norm: 82.087122
Active features: 113214
Line search trials: 2
Line search step: 0.440918
Seconds required for this iteration: 0.119

***** Iteration #124 *****
Loss: 1927.264797
Feature norm: 25.126066
Error norm: 44.290725
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.076

***** Iteration #125 *****
Loss: 1927.043198
Feature norm: 25.121461
Error norm: 20.584260
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.065

***** Iteration #126 *****
Loss: 1926.928727
Feature norm: 25.119187
Error norm: 25.983031
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.067

***** Iteration #127 *****
Loss: 1926.748524
Feature norm: 25.115867
Error norm: 32.831449
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seco

***** Iteration #166 *****
Loss: 1922.306722
Feature norm: 25.209980
Error norm: 11.892043
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.057

***** Iteration #167 *****
Loss: 1922.282004
Feature norm: 25.215572
Error norm: 9.607494
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.054

***** Iteration #168 *****
Loss: 1922.260967
Feature norm: 25.216390
Error norm: 7.358230
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.063

***** Iteration #169 *****
Loss: 1922.207633
Feature norm: 25.224609
Error norm: 9.612370
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.064

***** Iteration #170 *****
Loss: 1922.195428
Feature norm: 25.230022
Error norm: 15.372106
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds

***** Iteration #212 *****
Loss: 1921.809445
Feature norm: 25.346680
Error norm: 5.095186
Active features: 113214
Line search trials: 2
Line search step: 0.450932
Seconds required for this iteration: 0.107

***** Iteration #213 *****
Loss: 1921.805826
Feature norm: 25.346521
Error norm: 3.047257
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.052

***** Iteration #214 *****
Loss: 1921.802394
Feature norm: 25.346169
Error norm: 1.726218
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.062

***** Iteration #215 *****
Loss: 1921.801752
Feature norm: 25.345845
Error norm: 4.108159
Active features: 113214
Line search trials: 2
Line search step: 0.404072
Seconds required for this iteration: 0.108

***** Iteration #216 *****
Loss: 1921.800691
Feature norm: 25.345584
Error norm: 3.006990
Active features: 113214
Line search trials: 1
Line search step: 1.000000
Seconds r

In [191]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 0
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

boot (B)
time (I)
is (O)
super (O)
fast (O)
, (O)
around (O)
anywhere (O)
from (O)
35 (O)
seconds (O)
to (O)
1 (O)
minute (O)
. (O)


In [192]:

def  bio_classification_reportbio_clas (y_true, y_pred):

    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag)
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [193]:
print(bio_classification_reportbio_clas (y_test, y_pred))

             precision    recall  f1-score   support

          B       0.86      0.66      0.75       651
          I       0.90      0.53      0.67       408

avg / total       0.96      0.96      0.96     11800



In [194]:
aspects_arr =[]


for  i in range(len(X_test)):
    
    row = test[i]
    asp = []
    for j in range(len(row)):
        
        word = row[j][0]
        y = row[j][-1]
        y_hat = y_pred[i][j]
        if y_hat == "B":
            asp.append(word)

        elif y_hat == "I":

            if asp:
                asp[-1] += " "+word
            else:
                asp.append(word)


    aspects_arr.append(asp)
    

In [195]:
aspects_arr

[['Boot time'],
 ['tech support'],
 [],
 ['Set up'],
 ['Windows'],
 [],
 ['fan of click pads', 'speakers', 'price tag'],
 [],
 ['installation disk'],
 ['use'],
 ['Works', 'apple OS'],
 [],
 ['features'],
 ['speedy WiFi connection', 'battery life'],
 ['delete key'],
 [],
 [],
 [],
 [],
 [],
 ['track pad'],
 [],
 ['outstanding graphics'],
 ['mountain lion'],
 ['build', 'durability'],
 ['battery life'],
 [],
 [],
 [],
 ['Windows'],
 ['baterry'],
 [],
 ['size', 'weight'],
 ['Apples'],
 [],
 ['performance'],
 [],
 [],
 ['USB ports'],
 [],
 [],
 ['speed', 'screen'],
 [],
 ['price'],
 ['Hardware performance'],
 [],
 ['set up'],
 [],
 ['Keyboard', 'responds'],
 [],
 ['Windows 8'],
 [],
 [],
 [],
 [],
 [],
 ['Windows'],
 ['usb ports'],
 [],
 [],
 ['features', 'screen', 'keyboard'],
 [],
 ['Performance'],
 [],
 [],
 ['quality', 'performance'],
 ['OS'],
 [],
 ['portability'],
 [],
 [],
 ['Windows'],
 [],
 [],
 ['performance'],
 [],
 [],
 ['performance'],
 ['lit up keys', 'screen display'],
 [],
 

In [183]:
aspect_dic = {}
for asp in aspects_arr:
    for a in asp:
        if a not in aspect_dic:
            aspect_dic[a] = 1
        aspect_dic[a] += 1


In [196]:
## Top Aspects
sorted(aspect_dic.items(), key = lambda q: q[1],reverse=True)

[('price', 19),
 ('features', 14),
 ('performance', 14),
 ('OS', 13),
 ('Windows', 11),
 ('use', 9),
 ('size', 9),
 ('screen', 9),
 ('keyboard', 9),
 ('battery', 8),
 ('battery life', 7),
 ('speed', 7),
 ('design', 7),
 ('windows', 7),
 ('hardware', 6),
 ('USB ports', 5),
 ('quality', 5),
 ('operating system', 5),
 ('sound', 5),
 ('hard drive', 5),
 ('Windows 7', 5),
 ('gaming', 5),
 ('ports', 5),
 ('Mac OS', 5),
 ('Runs', 5),
 ('RAM', 5),
 ('speakers', 4),
 ('track pad', 4),
 ('set up', 4),
 ('Performance', 4),
 ('portability', 4),
 ('shipping', 4),
 ('mouse', 4),
 ('service', 4),
 ('built', 4),
 ('memory', 4),
 ('start up', 4),
 ('works', 4),
 ('software', 4),
 ('HDMI', 4),
 ('Set up', 3),
 ('price tag', 3),
 ('build', 3),
 ('durability', 3),
 ('weight', 3),
 ('Keyboard', 3),
 ('OSX', 3),
 ('colors', 3),
 ('apps', 3),
 ('specs', 3),
 ('customer service', 3),
 ('delivery', 3),
 ('games', 3),
 ('Fan', 3),
 ('switch sooner', 3),
 ('touchpad', 3),
 ('iPhoto', 3),
 ('word processing', 3),

In [198]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B      -> I       2.070752
O      -> O       1.817305
I      -> I       1.534651
O      -> B       1.313853
I      -> B       -0.527122
B      -> O       -0.697204
I      -> O       -1.044964
B      -> B       -1.362758
O      -> I       -3.104513

Top unlikely transitions:
B      -> I       2.070752
O      -> O       1.817305
I      -> I       1.534651
O      -> B       1.313853
I      -> B       -0.527122
B      -> O       -0.697204
I      -> O       -1.044964
B      -> B       -1.362758
O      -> I       -3.104513


In [199]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
0.860446 O      bias
0.833801 O      EOS2
0.785308 O      BOS1
0.727360 I      -2:word[:2]=vi
0.718837 I      -1:postag2=NN
0.629276 I      +2:word.lower=hook
0.619635 B      word.lower=storage
0.619635 B      -1:word.lower=storage
0.619635 B      +1:word.lower=storage
0.598670 B      word.lower=resolution
0.598670 B      -1:word.lower=resolution
0.598670 B      +1:word.lower=resolution
0.589084 I      -2:word[:3]=vir
0.572855 I      -2:word[-3:]=ses
0.562382 I      +2:word[:3]=hoo
0.557576 B      -1:word[:3]=cos
0.550392 I      +2:word[-3:]=ket
0.549927 B      word[:3]=cos
0.549927 B      +1:word[:3]=cos
0.544273 I      -2:word.lower=easy

Top negative:
-0.592668 O      word.lower=software
-0.592668 O      -1:word.lower=software
-0.592668 O      +1:word.lower=software
-0.593912 O      -1:word[-3:]=pad
-0.598482 I      -2:word[-3:]=ity
-0.610965 I      +2:word[:2]=ha
-0.614037 I      BOS2
-0.627525 O      +2:word.lower=hook
-0.644127 O      word.lower=performance
-0.64412